In [1]:
%matplotlib notebook
%reload_ext autoreload
%autoreload 2

import datetime
from os import path, environ
from copy import deepcopy
from multiprocessing import Pool

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split

from models import fit_composite_model
# source file, see docs/5-dataset.md for info on field names
chiller_file = path.join(environ['DATADIR'], 'EngineeringScienceBuilding', 'Chillers.csv')
plot_path = path.join('..', 'docs', 'img')

In [2]:
# Read pre-processed data
df = pd.read_csv(chiller_file, index_col='Time', parse_dates=True, dtype=float)
df.dropna(inplace=True)

# Evaporative Cooling Model

Modelling the relationship between the cooling done by the cooling tower and the environmental, system, and control inputs.

$$
\begin{align*}
T(t) &= T(0) e^{-\frac{k T_a v_f R}{T_w c_m m} t}
\end{align*}
$$

Where:

* $T(0)$ is the warm entering water temperature `TempCondOut`,
* $T(t)$ is the exiting cool water temperature `TempCondIn`,
* $T_a$ is ambient air temperature `TempAmbient`,
* $v_f$ is avarage fan speed `0.5 * (PerFreqFanA + PerFreqFanB)`,
* $R$ is solar irradiance - constant if assuming shade.
* $T_w$ is wet-bulb temperature `TempWetBulb`,
* $c_m$ is specific mass heat capacity of water,
* $m$ is the mass of water being cooled - constant if assuming steady flow rate.

The model predicts $T(t)$ from all other factors.

## Multi-Layer Perceptron

The motivation for an MLP model is the assumption that inputs manifest instantaneously as outputs with not delay. This is a simplification as water takes some time to cycle through the cooling tower.

The inputs are chosen assuming steady flow rate, constant irradiance.

The control variables are `PerFreqFanA` and `PerFreqFanB` which usually track each other. They are distributed bi-modally around 100% and 0% power with a minority of samples falling in between. A concern is that the model may learn to treat control inputs as constant. Two approaches are chosen:

* A single model is learned over all data,
* Samples are clustered by control=0, control >= 0.95, and 0 < control  < 0.95. A separate model is learned for each cluster.

In [3]:
# Data
feature_cols = ['TempCondOut', 'PerFreqFanA', 'PerFreqFanB', 'TempAmbient', 'TempWetBulb'] 
X, Y = df.loc[:, feature_cols], df['TempCondIn']
# setting up cluster selectors
c1 = df['PerFreqFanA'] >= 0.95
c0 = df['PerFreqFanA'] == 0 
cmid = ~ (c1 | c0)
# generating clusters
X1, Y1 = X[c1], Y[c1]
Xmid, Ymid = X[cmid], Y[cmid]
X0, Y0 = X[c0], Y[c0]
# generating training/testing sets for each cluster
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.1)
X1train, X1test, Y1train, Y1test = train_test_split(X1, Y1, test_size=0.1)
Xmidtrain, Xmidtest, Ymidtrain, Ymidtest = train_test_split(Xmid, Ymid, test_size=0.1)
X0train, X0test, Y0train, Y0test = train_test_split(X0, Y0, test_size=0.1)

### Single MLP

In [ ]:
# Pipeline
scaler = StandardScaler()              # scale to 0 mean and unit variance
regressor = MLPRegressor(hidden_layer_sizes=(20,20), 
                         learning_rate_init=1e-3,
                         verbose=True) # regression model
est = Pipeline([('scaler', scaler), ('regressor', regressor)])

In [ ]:
est.fit(Xtrain, Ytrain)

In [ ]:
print('Score on all control:\t{:.4f}'.format(est.score(Xtest, Ytest)))
print('Score on >95%-control:\t{:.4f}'.format(est.score(X0test, Y0test)))
print('Score on mid-control:\t{:.4f}'.format(est.score(X1test, Y1test)))
print('Score on 0%-control:\t{:.4f}'.format(est.score(Xmidtest, Ymidtest)))

### Composite MLP

In [ ]:
scaler = StandardScaler()              # scale to 0 mean and unit variance
regressor = MLPRegressor(hidden_layer_sizes=(20,20), 
                         learning_rate_init=1e-3,
                         verbose=False,
                         max_iter=1000,
                         solver='adam') # regression model
est = Pipeline([('scaler', scaler), ('regressor', regressor)])

est1, estmid, est0 = fit_composite_model(est, zip((X1train, Xmidtrain, X0train),
                                                  (Y1train, Ymidtrain, Y0train)))
print('Loss on >95%-control:\t{:.4f}'.format(est1.named_steps['regressor'].loss_))
print('Loss on mid-control:\t{:.4f}'.format(estmid.named_steps['regressor'].loss_))
print('Loss on 0%-control:\t{:.4f}'.format(est0.named_steps['regressor'].loss_))

In [25]:
score1 = est1.score(X1test, Y1test)
scoremid = estmid.score(Xmidtest, Ymidtest)
score0 = est0.score(X0test, Y0test)
scoreall = (len(X1test)*score1 + len(Xmidtest)*scoremid + len(X0test)*score0) / len(Xtest)
print('Score on all control:\t{:.4f}'.format(scoreall))
print('Score on >95%-control:\t{:.4f}'.format(score1))
print('Score on mid-control:\t{:.4f}'.format(scoremid))
print('Score on 0%-control:\t{:.4f}'.format(score0))

Score on all control:	0.8641
Score on >95%-control:	0.8700
Score on mid-control:	0.6506
Score on 0%-control:	0.9793
